In [1]:
from operator import and_, or_
from functools import reduce

import pathlib as pl
import pandas as pd
import numpy as np

from tinydb import TinyDB

from candas.test import QuantStudio
from candas.lims import Librarian, library
from candas.lims.schema import Fragment, Amplicon, Core, Primer, ProbeOligo, ProbeSeq

In [2]:
lib = Librarian().load(library).open()
# lib.get_by_name(Experiment, 'JG073A').wells[0].reaction

In [3]:
here = pl.Path.cwd()
data_pth = here / 'raw_data'

In [ ]:

data = pd.DataFrame()
meta = pd.DataFrame()
figs = pd.DataFrame()
smix = pd.DataFrame()

fragment_sequences = {}
primer_sequences = {}

In [5]:
seq_aliases = {
    'ISO': 'S044.12',
    'GC15': 'S044.9',
    'GC25': 'S044.10',
    'GC35': 'S044.11',
    'GC45': 'S044.12',
    'GC55': 'S044.14',
    'GC65': 'S044.15',
    'GC75': 'S044.16',
    'GC80': 'S044.17',
    'GC85': 'S044.18',
    'BP240': 'S044.25',
    'BP200': 'S044.24',
    'BP160': 'S044.21',
    'BP55': 'S044.8',
    'BP40': 'S044.4',
    'BP30': 'S044.2',
    'BP15': 'S044.0',
}

In [6]:
def name_reactions(df):
    return df.assign(
        Reaction=lambda df: df.apply(
            lambda row: f"{row.Experiment}_{row.Well:03d}", axis=1
        )
    )


def process_experiment(experiment, filename, cmax=50):
    qs = (
        QuantStudio(data_pth / filename, experiment)
        .import_data()
        .format_reactions()
        .index_reactions()
        .subtract_background()
        .normalize_reactions(cmax=cmax)
        .extract_endpoints(cmax=cmax, name="FAM-HEX")
    )
    wide = name_reactions(qs.reactions.wide)
    wide["Endpoint"] = wide.apply(lambda row: row.Fluorescence[cmax - 1], axis=1)
    return wide[
        [
            "Reaction",
            "Experiment",
            "Well",
            "Target",
            "Copies",
            "Reporter",
            "Sample",
            "CT",
            "Endpoint",
            "Cycle",
            "Fluorescence",
            "Corr_Fluorescence",
        ]
    ]


def parse_rxn_specs(filename, id_column="Sample"):
    db_file = data_pth / filename
    rxn_db = TinyDB(db_file)
    rxns = pd.DataFrame(
        [rxn["oligos"] | {id_column: rxn["name"]} for rxn in rxn_db.all()]
    )
    rxns = rxns.melt(
        id_vars=[id_column], var_name="component", value_name="concentration"
    ).assign(units="copies")
    return rxns

def extract_meta(data):
    meta = data[['Reaction','Target','Copies']].rename(
        columns={"Target": "component", "Copies": "concentration"}
    ).assign(units="copies", component=lambda df: df.component.replace(seq_aliases))
    return neaten_meta(meta)

def neaten_meta(meta):
    return meta.drop_duplicates().reset_index(drop=True)[
        ["Reaction", "component", "concentration", "units"]
    ]


def append_data(new_data):
    assert new_data.shape[0] > 0
    new = (
        pd.concat(
            [
                data,
                new_data[
                    [
                        "Reaction",
                        "Experiment",
                        "Well",
                        "Reporter",
                        "CT",
                        "Endpoint",
                        "Cycle",
                        "Fluorescence",
                        "Corr_Fluorescence",
                    ]
                ],
            ],
            axis=0,
            ignore_index=True,
        )
        .drop_duplicates(subset=["Reaction", "Experiment", "Well", "Reporter"])
        .reset_index(drop=True)
    )

    print(f"The shape of `data` is now {new.shape[0]} x {new.shape[1]}")
    return new


def append_meta(new_meta, supermix=None):
    assert new_meta.shape[0] > 0

    if supermix is not None:
        assert isinstance(supermix, str)
        if not supermix.endswith("_supermix"):
            supermix = f"{supermix}_supermix"
            
        new_meta = pd.concat(
            [
                new_meta,
                pd.DataFrame(new_meta.Reaction.unique(), columns=["Reaction"]).assign(
                    component=supermix,
                    concentration=1,
                    units="X",
                ),
            ]
        )

    new = (
        pd.concat(
            [
                meta,
                new_meta[["Reaction", "component", "concentration", "units"]],
            ],
            axis=0,
            ignore_index=True,
        )
        .drop_duplicates()
        .replace({"component": seq_aliases})
        .reset_index(drop=True)
    )

    print(f"The shape of `meta` is now {new.shape[0]} x {new.shape[1]}")
    return new


def append_smix(new_smix, name):
    assert new_smix.shape[0] > 0
    if not new_smix.component.str.contains("Master Mix").any():
        new_smix.loc[-1] = ["TaqMan Fast Advanced Master Mix", 1, "X"]
    new_smix.reset_index(drop=True, inplace=True)

    assert isinstance(name, str)
    if not name.endswith("_supermix"):
        name = f"{name}_supermix"
        
    new = (
        pd.concat(
            [
                smix,
                new_smix.assign(SuperMix=name)[
                    ["SuperMix", "component", "concentration", "units"]
                ],
            ],
            axis=0,
            ignore_index=True,
        )
        .drop_duplicates()
        .reset_index(drop=True)
    )

    print(f"The shape of `smix` is now {new.shape[0]} x {new.shape[1]}")
    return new


def append_figs(reactions, alias):
    new_figs = pd.DataFrame(reactions, columns=["Reaction"]).assign(figure_alias=alias)
    new = (
        pd.concat([figs, new_figs], axis=0, ignore_index=True)
        .drop_duplicates()
        .reset_index(drop=True)
    )

    print(f"The shape of `figs` is now {new.shape[0]} x {new.shape[1]}")
    return new


def filter_reactions(this, this_filter):

    these_rxns = this.groupby("Reaction").filter(this_filter).Reaction.unique()

    n_rxns = len(these_rxns)
    assert n_rxns > 0
    print(n_rxns, "matching reactions")

    return these_rxns

# Fig 3 wool_wren

## JG034: Bipartite

In [7]:
primers_034 = [
    ['FP004', 125, 'nM'],
    ['RP004', 125, 'nM'],
]

probes_034 = [
    ['Pr002', 250, 'nM'],
    ['Pr003', 250, 'nM'],]

plastic_yak_smix = pd.DataFrame(
    [[seq_aliases['GC55'], 10**5, 'copies']] + probes_034 + primers_034,
    columns=["component", "concentration", "units"],
)
smix = append_smix(plastic_yak_smix, "plastic_yak")

The shape of `smix` is now 6 x 4


In [8]:
JG034_data = process_experiment('JG034', 'JG034 TMCC1 Gen2 Competitors - 59C v3.xlsx')

JG034_data = JG034_data[JG034_data.Target.str.contains("GC55")].assign(Target='S044.13')

JG034_meta = pd.DataFrame()
JG034_meta['Reaction'] = JG034_data.Reaction
JG034_meta['component'] = JG034_data.Target.str.split(" - ", expand=True)[0]
JG034_meta['concentration'] = JG034_data.Copies
JG034_meta['units'] = 'copies'
JG034_meta = neaten_meta(JG034_meta)
JG034_rxns = JG034_meta.Reaction.unique()

data = append_data(JG034_data)
meta = append_meta(JG034_meta, supermix='plastic_yak')
figs = append_figs(JG034_rxns, 'plastic_yak')
figs = append_figs(JG034_rxns, 'ceramic_zebu')

The shape of `data` is now 28 x 9
The shape of `meta` is now 28 x 4
The shape of `figs` is now 14 x 2
The shape of `figs` is now 28 x 2


## JG071B: Tripartite

In [9]:
primers_071b = [
    ["FP001", 100, "nM"],
    ["RP001", 100, "nM"],
    ["FP005", 100, "nM"],
    ["RP005", 100, "nM"],
]

probes_071b = [
    ["Pr002", 200, "nM"],
    ["Pr003", 200, "nM"],
]

metal_anchovy_smix = pd.DataFrame(
    [["S057.3.2", 10**6, "copies"], ["S057.4.2", 10**4, "copies"]]
    + probes_071b
    + primers_071b,
    columns=["component", "concentration", "units"],
)
smix = append_smix(metal_anchovy_smix, "metal_anchovy")

The shape of `smix` is now 15 x 4


In [10]:
JG071B_data = process_experiment('JG071B', "JG071B Tripartite sweep GBP6.xlsx")

JG071B_meta = parse_rxn_specs("JG071B Reaction Specifications.json")
JG071B_meta = neaten_meta(JG071B_meta.merge(JG071B_data[['Reaction','Sample']]))

def this_filter(grp):
    conditions = [
        (grp.component == "S036.5") & (grp.concentration > 10),
        (grp.component == "S057.3.2") & (grp.concentration == 10**6),
        (grp.component == "S057.4.2") & (grp.concentration == 10**4)
    ]

    return reduce(and_, [cond.any() for cond in conditions])

JG071B_rxns = filter_reactions(JG071B_meta, this_filter)
JG071B_data = JG071B_data[JG071B_data.Reaction.isin(JG071B_rxns)]
JG071B_meta = JG071B_meta[JG071B_meta.Reaction.isin(JG071B_rxns)]
JG071B_meta = JG071B_meta[JG071B_meta.component == 'S036.5']

data = append_data(JG071B_data)
meta = append_meta(JG071B_meta, supermix='metal_anchovy')
figs = append_figs(JG071B_rxns, "metal_anchovy")
figs = append_figs(JG071B_rxns, "digital_bear")

13 matching reactions
The shape of `data` is now 54 x 9
The shape of `meta` is now 54 x 4
The shape of `figs` is now 41 x 2
The shape of `figs` is now 54 x 2


## JG074C: Antiparallel Redundant Module

In [11]:
primers_074C = [
    ["FP001", 100, "nM"],
    ["RP001", 100, "nM"],
    ["FP002", 100, "nM"],
    ["RP002", 100, "nM"],
]

probes_074C = [
    ["Pr001", 200, "nM"],
    ["Pr004", 200, "nM"],
    ["Pr007", 200, "nM"],
    ["Pr008", 200, "nM"],
]

electric_chinchilla_smix = pd.DataFrame(
    [["S037.01.01b", 10**6, "copies"], ["S036.1b", 10**4, "copies"]]
    + probes_074C
    + primers_074C,
    columns=["component", "concentration", "units"],
)
smix = append_smix(electric_chinchilla_smix, "electric_chinchilla")

The shape of `smix` is now 26 x 4


In [12]:
JG074C_data = process_experiment('JG074C', "JG074C_L2_500_competitor_sweep.xlsx")

JG074C_meta = parse_rxn_specs("JG074C Reaction Specifications.json")
JG074C_meta = neaten_meta(JG074C_meta.merge(JG074C_data[['Reaction','Sample']]))

def this_filter(grp):
    conditions = [
        (grp.component == "S037.01.01b") & (grp.concentration == 10**6),
        (grp.component == "S036.1b") & (grp.concentration == 10**2)
    ]

    return reduce(and_, [cond.any() for cond in conditions])

JG074C_rxns = filter_reactions(JG074C_meta, this_filter)
JG074C_data = JG074C_data[JG074C_data.Reaction.isin(JG074C_rxns)]
JG074C_meta = JG074C_meta[JG074C_meta.Reaction.isin(JG074C_rxns)]
JG074C_meta = JG074C_meta[JG074C_meta.component == 'L2_500']

data = append_data(JG074C_data)
meta = append_meta(JG074C_meta, supermix='electric_chinchilla')
figs = append_figs(JG074C_rxns, "electric_chinchilla")
figs = append_figs(JG074C_rxns, "solar_dog")

16 matching reactions
The shape of `data` is now 86 x 9
The shape of `meta` is now 86 x 4
The shape of `figs` is now 70 x 2
The shape of `figs` is now 86 x 2


## JG074D: Parallel Redundant Module

In [13]:
primers_074D = [
    ["FP001", 100, "nM"],
    ["RP001", 100, "nM"],
    ["FP002", 100, "nM"],
    ["RP002", 100, "nM"],
]

probes_074D = [
    ["Pr001", 200, "nM"],
    ["Pr004", 200, "nM"],
    ["Pr007", 200, "nM"],
    ["Pr008", 200, "nM"],
]

lunar_earwig_smix = pd.DataFrame(
    [["S037.01.01b", 10**8, "copies"], ["S036.0", 10**2, "copies"]]
    + probes_074D
    + primers_074D,
    columns=["component", "concentration", "units"],
)
smix = append_smix(lunar_earwig_smix, "lunar_earwig")

The shape of `smix` is now 37 x 4


In [14]:
JG074D_data = process_experiment('JG074D', "JG074D_L2_500_like_competitor_sweep.xlsx")

JG074D_meta = parse_rxn_specs("JG074D Reaction Specifications.json")
JG074D_meta = neaten_meta(JG074D_meta.merge(JG074D_data[['Reaction','Sample']]))

def this_filter(grp):
    conditions = [
        (grp.component == "S037.01.01b") & (grp.concentration == 10**8),
        (grp.component == "S036.0") & (grp.concentration == 10**2)
    ]

    return reduce(and_, [cond.any() for cond in conditions])

JG074D_rxns = filter_reactions(JG074D_meta, this_filter)
JG074D_data = JG074D_data[JG074D_data.Reaction.isin(JG074D_rxns)]
JG074D_meta = JG074D_meta[JG074D_meta.Reaction.isin(JG074D_rxns)]
JG074D_meta = JG074D_meta[JG074D_meta.component == 'L2_500_like']

data = append_data(JG074D_data)
meta = append_meta(JG074D_meta, supermix="lunar_earwig")
figs = append_figs(JG074D_rxns, "lunar_earwig")
figs = append_figs(JG074D_rxns, "cosmic_falcon")

15 matching reactions
The shape of `data` is now 116 x 9
The shape of `meta` is now 116 x 4
The shape of `figs` is now 101 x 2
The shape of `figs` is now 116 x 2


# Fig 4 galactic_gorilla

In [15]:
ARG1_wildtype = 'S036.0'
GBP6_wildtype = 'S036.5'
TMCC1_wildtype = 'S044.13'
PRDM1_wildtype = 'S_PRDM1_WTb'

ARG1_competitor = 'S036.1b'
GBP6_competitor = 'S044.6'
TMCC1_competitor = 'S037.01.01b'
PRDM1_competitor = 'S057.5.7'

ARG1_comp_lg10_copies = 1.4
GBP6_comp_lg10_copies = 1.0
TMCC1_comp_lg10_copies = 1.8
PRDM1_comp_lg10_copies = 3.8

ARG1_p1 = 'FP002'
ARG1_p2 = 'FP002'
GBP6_p1 = 'FP001'
GBP6_p2 = 'RP001'
TMCC1_p1 = 'FP004'
TMCC1_p2 = 'RP004'
PRDM1_p1 = 'FP003'
PRDM1_p2 = 'RP008'

ARG1_pnM = 100
GBP6_pnM = 80
TMCC1_pnM = 80
PRDM1_pnM = 40

galactic_gorilla_probes = ['Pr001','Pr002','Pr003','Pr004','Pr005','Pr007','Pr008','Pr009']



In [16]:
for frag, name in zip([
    ARG1_wildtype,
    GBP6_wildtype,
    TMCC1_wildtype,
    PRDM1_wildtype,
    ARG1_competitor,
    GBP6_competitor,
    TMCC1_competitor,
    PRDM1_competitor,
],[
    'ARG1_wildtype',
    'GBP6_wildtype',
    'TMCC1_wildtype',
    'PRDM1_wildtype',
    'ARG1_competitor',
    'GBP6_competitor',
    'TMCC1_competitor',
    'PRDM1_competitor'
]):
    print(f"Fragment: {frag} ({name})")
    try:
        f = lib.get_by_name(Fragment, frag)
    except Exception as e:
        print(f"Fragment {frag} not found in LIMS")
        continue
    
    print(f'Probes: {f.amplicon.core.probe.name}')
    print()

Fragment: S036.0 (ARG1_wildtype)
Probes: Pr004

Fragment: S036.5 (GBP6_wildtype)
Probes: Pr001

Fragment: S044.13 (TMCC1_wildtype)
Probes: Pr002

Fragment: S_PRDM1_WTb (PRDM1_wildtype)
Probes: Pr005

Fragment: S036.1b (ARG1_competitor)
Probes: Pr008

Fragment: S044.6 (GBP6_competitor)
Probes: Pr003

Fragment: S037.01.01b (TMCC1_competitor)
Probes: Pr007

Fragment: S057.5.7 (PRDM1_competitor)
Fragment S057.5.7 not found in LIMS


In [17]:
galactic_gorilla_smix = pd.DataFrame(
    data=[
        [ARG1_competitor, int(10**ARG1_comp_lg10_copies), "copies"],
        [GBP6_competitor, int(10**GBP6_comp_lg10_copies), "copies"],
        [TMCC1_competitor, int(10**TMCC1_comp_lg10_copies), "copies"],
        [PRDM1_competitor, int(10**PRDM1_comp_lg10_copies), "copies"],
        [ARG1_p1, ARG1_pnM, "nM"],
        [ARG1_p2, ARG1_pnM, "nM"],
        [GBP6_p1, GBP6_pnM, "nM"],
        [GBP6_p2, GBP6_pnM, "nM"],
        [TMCC1_p1, TMCC1_pnM, "nM"],
        [TMCC1_p2, TMCC1_pnM, "nM"],
        [PRDM1_p1, PRDM1_pnM, "nM"],
        [PRDM1_p2, PRDM1_pnM, "nM"],
    ]
    + [[p, 200, "nM"] for p in galactic_gorilla_probes],
    columns=["component", "concentration", "units"],
)

smix = append_smix(galactic_gorilla_smix, "galactic_gorilla")

The shape of `smix` is now 57 x 4


In [18]:
JG069J_data = process_experiment("JG069J", "JG069J Final TB Experiment.xlsx").drop(
    columns=["Target", "Copies", "Sample"]
)

JG069J_meta = (
    pd.read_csv(data_pth / "JG069J Plate Map.csv")[
        ["PRDM1", "TMCC1", "GBP6", "ARG1", "Well"]
    ]
    .melt(id_vars=["Well"], var_name="component", value_name="log10_copies")
    .replace(
        {
            "component": {
                "PRDM1": PRDM1_wildtype,
                "TMCC1": TMCC1_wildtype,
                "GBP6": GBP6_wildtype,
                "ARG1": ARG1_wildtype,
            }
        }
    )
    .assign(
        concentration=lambda df: 10 ** df.log10_copies.astype(int),
        units="copies",
        Reaction=lambda df: df.apply(lambda row: f"JG069J_{row.Well:03d}", axis=1),
    )
    .drop(columns=["log10_copies", "Well"])
)


JG069J_rxns = pd.DataFrame(JG069J_data.Reaction.unique(), columns=["Reaction"])
JG069J_meta = pd.concat(
    [
        JG069J_meta,
        pd.DataFrame(JG069J_data.Reaction.unique(), columns=["Reaction"]).assign(
            component="galactic_gorilla_supermix",
            concentration=1,
            units="X",
        ),
    ]
)

meta = append_meta(JG069J_meta, supermix="galactic_gorilla")
data = append_data(JG069J_data)
figs = append_figs(JG069J_data.Reaction.unique(), "galactic_gorilla")

The shape of `meta` is now 1916 x 4
The shape of `data` is now 820 x 9
The shape of `figs` is now 468 x 2


In [19]:
S057_5_7_seq = 'GCATCAACCTGTACTCATCTTACGTCACCCTATTCCTTTCATTGTTTCTAGCACCGTCTTAATCTACACCGCACTGCCCAGGAAGAACTATCGAGAAAGCGCCGTAGTGTTGTGATTTGTTGGGTGGAGAAAAGCCACATGAATGCAAAATAAATACTAAACAAAACTAACAACACAAAAATGGAACACGGCCTTTGACAGTGATAAACTGGAGAGAAACCATACCAATGGGTCGTGGGCTATAACTTTCCTACTTACATTACGGATTGAAACGGGATGGGTCTGTTGGTGCTCGAATACAGATA'
L2_500_seq = 'TTTGGTTTGGCTTATGTGGGTTCGGGCAAGAACCAAGACCCTCAGGGAGGGAATCACAGTCACTGGGAATCGTCTGGGAACTCTGGCAGTGACTTATGTAGAGGCCATCAACAGTGGAGCAGTGCCTTGTCTGGAGAATGTATTGAGGGCATACGCAGCACGACCATGCACGTAATCCTAATTAACACCCGCAGATATTTGGGATAGCGGCGTTGTCTGTTTTACGGACGTAAGCATGACACAATTCGACCGGCAACGTGGCTGAACCAATGTGAGAGAATAAATGTGCGTGTGCAAATTACTAATCAAGAAAAGTAGCAGGGAGAGGTATACGGCTCACTTCTATAGCCCATCGCGGACTTATCCGACGCTATATAGGGGAACGCACTACCAGCTTCCAGGTACGTACTATCTTCTATGGACAACCGCCTAGTTCTTAACACTCCAGGAGGCAACTTCAGTACATAGTGGTACGCGCTACGGCGGATGCTTATTTCGATGAGCCTGACAATTAACGGGCAATCAAAGTCGCTTGTTGAAGTAATACATGCGGTGGAGTCTGTCCTGCCAGATCGAGGTCAACAAGAGTTTATATGGCCATTGAATGCCTTGAATGTTCACGCTTAAAGACGCTCCCAACGCCAAGTCCAGAACCATAGGGATTATTGGAGCTCCTTTCTCAAAGGGACAGCCACGAGGAGGGGTGGAAGAAGGCCCTACAGTATTGAGAAAGGCTGGTCTGCTTGAGAAACTTAAAGAACAAGAGTGTGATGTGAAGGATTATGGGGATAGTTTGAGTTCGATCCAGACAGCC'
L2_500_like_seq = 'TTTGGTTTGGCTTATGTGGGTTCGGGCAAGAACCAAGACCCTCAGGGAGGGAATCACAGTCACTGGGAATCGTCTGGGAACTCTGGCAGTGACTTATGTAGAGGCCATCAACAGTGGAGCAGTGCCTTGTCTGGAGAATGTATTGAGGGCATACGCAGCACGACCATGCACGTAATCCTAATTAACACCCGCAGATATTTGGGATAGCGGCGTTGTCTGTTTTACGGACGTAAGCATGACACAATTCGACCGGCAACGTGGCTGAACCAATGTGAGAGAATAAATGTGCGTGTGCAAATTACTAATCAAGAAAAGTAGCAGGGAGAGGTATACGGCTCACTTCTATAGCCCATCGCGGACTTATCCGACGCTATATAGGGGAACGCACTACCAGCTTCCAGGTACGTACTATCTTCTATGGACAACCGCCTAGTTCTTAACACTCCAGGAGGCAACTTCAGTACATAGTGGTACGCGCTACGGCGGATGCTTATTTCGATGAGCCTGACAATTAACGGGCAATCAAAGTCGCTTGTTGAAGTAATACATGCGGTGGAGTCTGTCCTGCCAGATCGAGGTCAACAAGAGTTTATATGGCCATTGAATGCCTTGAATGTTCACGCTTAAAGACGCTCCCAACGCCAAGTCCAGAACCATAGGGATTATTGGAGCTCCTTTCTCAAAGGGACAGCCACGAGGAGGGGTGGAAGAAGGCCCTACAGTATTTACAGGCTGGGCCAAATGTCAGAAAATTAAAGAACAAGAGTGTGATGTGAAGGATTATGGGGATAGTTTGAGTTCGATCCAGACAGCC'

fragment_sequences['S057.5.7'] = S057_5_7_seq
fragment_sequences['L2_500'] = L2_500_seq
fragment_sequences['L2_500_like'] = L2_500_like_seq


In [20]:
FP003 = lib.get_by_name(Primer, 'FP003')
RP008 = lib.get_by_name(Primer, 'RP008')
Pr009 = lib.get_by_name(ProbeSeq, 'Pr009')

A057_5_7_start = S057_5_7_seq.find(FP003.sequence)
C057_5_7_start = A057_5_7_start+len(FP003.sequence)
C057_5_7_end = S057_5_7_seq.find(RP008.reverse_complement)
A057_5_7_end = C057_5_7_end+len(RP008.reverse_complement)

C057_5_7_seq = S057_5_7_seq[C057_5_7_start:C057_5_7_end]
A057_5_7_seq = S057_5_7_seq[A057_5_7_start:A057_5_7_end]

C057_5_7 = Core(
    name="C057.5.7",
    sequence=C057_5_7_seq,
    probe = Pr009,
)
A057_5_7 = Amplicon(
    name="A057.5.7",
    sequence=A057_5_7_seq,
    core=C057_5_7,
    primers = [FP003, RP008],
)
S057_5_7 = Fragment(
    name="S057.5.7",
    amplicons=[A057_5_7],
    sequence=S057_5_7_seq,
)

/home/john/projects/can-manuscript/env/candas/candas/lims/schema.py:907: UserWarning: No flanks specified; this is unusual.
  warn("No flanks specified; this is unusual.")


In [21]:
for probe in ['Pr001','Pr002','Pr003','Pr004','Pr005','Pr006','Pr007','Pr008','Pr009']:
    p = lib.get_by_name(ProbeSeq, probe)
    for seq, name in zip([L2_500_seq, L2_500_like_seq], ['L2_500','L2_500_like']):
        if p.sequence in seq:
            print(f"{p.name} found in {name}")
        elif p.reverse_complement in seq:
            print(f"{p.name} found in {name} (reverse complement)")

Pr001 found in L2_500 (reverse complement)
Pr001 found in L2_500_like (reverse complement)
Pr004 found in L2_500 (reverse complement)
Pr008 found in L2_500_like (reverse complement)


In [22]:
Pr001 = lib.get_by_name(ProbeSeq, 'Pr001')
Pr002 = lib.get_by_name(ProbeSeq, 'Pr002')
Pr003 = lib.get_by_name(ProbeSeq, 'Pr003')
Pr004 = lib.get_by_name(ProbeSeq, 'Pr004')
Pr005 = lib.get_by_name(ProbeSeq, 'Pr005')
Pr006 = lib.get_by_name(ProbeSeq, 'Pr006')
Pr007 = lib.get_by_name(ProbeSeq, 'Pr007')
Pr008 = lib.get_by_name(ProbeSeq, 'Pr008')
Pr009 = lib.get_by_name(ProbeSeq, 'Pr009')
Pr005.sequence in S057_5_7_seq

False

In [23]:
for f in [c for c in meta.component.unique() if 'supermix' not in c and 'P0' not in c]:
    try:
        lib.get_by_name(Fragment, f)
    except Exception as e:
        print(f"Failed to find {f} in LIMS")
        continue

Failed to find L2_500 in LIMS
Failed to find L2_500_like in LIMS


In [24]:
from candas.lims.schema import ProbeSeq

lib.get_by_name(ProbeSeq, 'Pr008').cores

[<Core(id=181, name=C_IFI44L_WT_alt, BP=92, GC=0.489)>,
 <Core(id=182, name=C_IFI44L_WT, BP=58, GC=0.500)>,
 <Core(id=183, name=C036.1b, BP=108, GC=0.472)>]

# Fig S2 sharp_jellyfish

In [25]:
JG063A_data = process_experiment(
    "JG063A", "JG063A Competitor Concentration Sweep.xlsx"
).assign(Target=lambda df: df.Target.str.split("_").str[0])
JG063B_data = process_experiment(
    "JG063B", "JG063B Competitor concentration sweep 2.xlsx"
).assign(Target=lambda df: df.Target.str.split("_").str[0])
JG063I_data = process_experiment("JG063I", "JG063I Primer Probe Sweep 5 logs.xlsx")
JG034_data = process_experiment(
    "JG034", "JG034 TMCC1 Gen2 Competitors - 59C v3.xlsx"
).assign(Target=lambda df: df.Target.str.split(" - ").str[0])

for qs in [
    JG063A_data,
    JG063B_data,
    JG063I_data,
    JG034_data,
]:
    qs.drop(qs.index[qs.Well == 281], inplace=True)
    
JG063I_primerprobe = (
    JG063I_data.Sample.str.removesuffix(" primer+probe")
    .str.split("+", expand=True)
    .rename(columns={0: "Primer nM", 1: "Probe nM"})
    .astype(int)
)

JG063I_data = JG063I_data.merge(JG063I_primerprobe, left_index=True, right_index=True)
JG063I_data = JG063I_data[JG063I_data["Probe nM"] == 200]

In [26]:
primers_063 = [
    ['FP004', 100, 'nM'],
    ['RP004x', 100, 'nM'],
]

probes_063 = [
    ['Pr002', 200, 'nM'],
    ['Pr003', 200, 'nM'],]

primers_034 = [
    ['FP004', 125, 'nM'],
    ['RP004', 125, 'nM'],
]

probes_034 = [
    ['Pr002', 250, 'nM'],
    ['Pr003', 250, 'nM'],]

dull_koala_smix = pd.DataFrame(
    probes_063 + primers_063,
    columns=["component", "concentration", "units"],
)
smix = append_smix(dull_koala_smix, "dull_koala")

bright_lizard_i_smix = pd.DataFrame(
    [['S056.4.3', 10**6, 'copies']] + probes_063 + primers_063,
    columns=["component", "concentration", "units"],
)
smix = append_smix(bright_lizard_i_smix, "bright_lizard_i")

bright_lizard_ii_smix = pd.DataFrame(
    [['S056.4.1', 10**4, 'copies']] + probes_063 + primers_063,
    columns=["component", "concentration", "units"],
)
smix = append_smix(bright_lizard_ii_smix, "bright_lizard_ii")

bright_lizard_iii_smix = pd.DataFrame(
    [[seq_aliases['GC55'], 10**5, 'copies']] + probes_034 + primers_034,
    columns=["component", "concentration", "units"],
)
smix = append_smix(bright_lizard_iii_smix, "bright_lizard_iii")

bright_lizard_iv_smix = pd.DataFrame(
    [['S056.4.4', 10**5, 'copies']] + probes_063 + primers_063,
    columns=["component", "concentration", "units"],
)  # wt 'S056.4.12'
smix = append_smix(bright_lizard_iv_smix, "bright_lizard_iv")

bright_lizard_v_smix = pd.DataFrame(
    [[seq_aliases['GC15'], 10**5, 'copies']] + probes_034 + primers_034,
    columns=["component", "concentration", "units"],
)
smix = append_smix(bright_lizard_v_smix, "bright_lizard_v")

The shape of `smix` is now 62 x 4
The shape of `smix` is now 68 x 4
The shape of `smix` is now 74 x 4
The shape of `smix` is now 80 x 4
The shape of `smix` is now 86 x 4
The shape of `smix` is now 92 x 4


In [27]:
dull_koala_data = JG063A_data.query('Sample == "S056.4.2_vs_S056.4.12"')
dull_koala_meta = extract_meta(dull_koala_data)


data = append_data(dull_koala_data)
meta = append_meta(dull_koala_meta, supermix="dull_koala")
figs = append_figs(dull_koala_data.Reaction.unique(), "dull_koala")

The shape of `data` is now 980 x 9
The shape of `meta` is now 2156 x 4
The shape of `figs` is now 548 x 2


In [28]:
bright_lizard_data = pd.concat(
    [
        JG063A_data,
        JG063B_data,
        JG034_data,
    ]
)

def bright_lizard_filter(grp):
    conditions = [
        (grp.Experiment == 'JG063A') & (grp.Reporter == 'HEX') & (grp.Copies == 10**5) & (grp.Sample == "S056.4.4_vs_S056.4.12"),
        (grp.Experiment == 'JG063B') & (grp.Reporter == 'HEX') & (grp.Copies == 10**4) & (grp.Sample == "S056.4.1_vs_S056.4.12"),
        (grp.Experiment == 'JG063B') & (grp.Reporter == 'HEX') & (grp.Copies == 10**6) & (grp.Sample == "S056.4.3_vs_S056.4.12"),
        (grp.Experiment == 'JG034') & (grp.Reporter == 'HEX')
    ]
    
    bright_lizard_targets = ["S056.4.3", "S056.4.1", "GC55", "S056.4.4", "GC15"]
    must_have = (grp.Reporter == 'FAM') & (grp.Copies > 10**1)

    return reduce(or_, [cond.any() for cond in conditions]) & must_have.any() & grp.Target.isin(bright_lizard_targets).any()


bright_lizard_rxns = filter_reactions(bright_lizard_data, bright_lizard_filter)

bright_lizard_data = bright_lizard_data[bright_lizard_data.Reaction.isin(bright_lizard_rxns)]
bright_lizard_i_data = bright_lizard_data.query("Target == 'S056.4.3'").assign(Target='S056.4.12')
bright_lizard_ii_data = bright_lizard_data.query("Target == 'S056.4.1'").assign(Target='S056.4.12')
bright_lizard_iii_data = bright_lizard_data.query("Target == 'GC55'").assign(Target='S044.13')
bright_lizard_iv_data = bright_lizard_data.query("Target == 'S056.4.4'").assign(Target='S056.4.12')
bright_lizard_v_data = bright_lizard_data.query("Target == 'GC15'").assign(Target='S044.13')

# bright_lizard_data.loc[bright_lizard_data.Experiment == 'JG034', 'Target'] = 'S044.13'
bright_lizard_i_meta = extract_meta(bright_lizard_i_data)
bright_lizard_ii_meta = extract_meta(bright_lizard_ii_data)
bright_lizard_iii_meta = extract_meta(bright_lizard_iii_data)
bright_lizard_iv_meta = extract_meta(bright_lizard_iv_data)
bright_lizard_v_meta = extract_meta(bright_lizard_v_data)

data = append_data(bright_lizard_i_data)
data = append_data(bright_lizard_ii_data)
data = append_data(bright_lizard_iii_data)
data = append_data(bright_lizard_iv_data)
data = append_data(bright_lizard_v_data)
meta = append_meta(bright_lizard_i_meta, supermix="bright_lizard_i")
meta = append_meta(bright_lizard_ii_meta, supermix="bright_lizard_ii")
meta = append_meta(bright_lizard_iii_meta, supermix="bright_lizard_iii")
meta = append_meta(bright_lizard_iv_meta, supermix="bright_lizard_iv")
meta = append_meta(bright_lizard_v_meta, supermix="bright_lizard_v")
figs = append_figs(bright_lizard_data.Reaction.unique(), "bright_lizard")

70 matching reactions
The shape of `data` is now 994 x 9
The shape of `data` is now 1008 x 9
The shape of `data` is now 1008 x 9
The shape of `data` is now 1022 x 9
The shape of `data` is now 1050 x 9
The shape of `meta` is now 2184 x 4
The shape of `meta` is now 2212 x 4
The shape of `meta` is now 2226 x 4
The shape of `meta` is now 2254 x 4
The shape of `meta` is now 2282 x 4
The shape of `figs` is now 618 x 2


In [29]:
probes_063I = [
    ['Pr002', 200, 'nM'],
    ['Pr003', 200, 'nM'],]

dim_mongoose_smix = pd.DataFrame(
    [['S056.1.1', 10**5, 'copies']] + probes_063I,
    columns=["component", "concentration", "units"],
)

JG063I_primer_nMs = JG063I_data[['Reaction','Primer nM']].rename(columns={'Primer nM':'concentration'}).assign(units='nM').drop_duplicates()
JG063I_primer_nMs = pd.concat(
    [
        JG063I_primer_nMs.assign(component='FP001'),
        JG063I_primer_nMs.assign(component='RP001x'),
    ],
    ignore_index=True,
)


JG063I_meta = extract_meta(JG063I_data).query("component != 'S056.1.1'")
JG063I_meta = pd.concat(
    [
        JG063I_meta, JG063I_primer_nMs
    ],
    ignore_index=True,
)

JG063I_rxns = JG063I_meta.Reaction.unique()

smix = append_smix(dim_mongoose_smix, "dim_mongoose")
meta = append_meta(JG063I_meta, supermix="dim_mongoose")
data = append_data(JG063I_data)
figs = append_figs(JG063I_rxns, "dim_mongoose")

The shape of `smix` is now 96 x 4
The shape of `meta` is now 2602 x 4
The shape of `data` is now 1210 x 9
The shape of `figs` is now 698 x 2


In [30]:
meta[meta.component == 'dim_mongoose_supermix']

,Reaction,component,concentration,units
2522,JG063I_005,dim_mongoose_supermix,1.0,X
2523,JG063I_010,dim_mongoose_supermix,1.0,X
2524,JG063I_015,dim_mongoose_supermix,1.0,X
2525,JG063I_020,dim_mongoose_supermix,1.0,X
2526,JG063I_024,dim_mongoose_supermix,1.0,X
...,...,...,...,...
2597,JG063I_365,dim_mongoose_supermix,1.0,X
2598,JG063I_370,dim_mongoose_supermix,1.0,X
2599,JG063I_375,dim_mongoose_supermix,1.0,X
2600,JG063I_380,dim_mongoose_supermix,1.0,X


In [31]:
all_components = meta.component.unique().tolist() + smix.component.unique().tolist()
all_components = [c for c in all_components if 'mix' not in c.lower()]
primers = [c for c in all_components if c.startswith('FP') or c.startswith('RP')]
probes = [c for c in all_components if c.startswith('Pr')]
fragments = [c for c in all_components if c not in primers and c not in probes]

for f in fragments:
    
    
    try:
        lib.get_by_name(Fragment, f)
    except Exception as e:
        try:
            fragment_sequences[f]
        except KeyError as e:
            print(f"Failed to find {f} in LIMS or dictionary")
        continue
    
for p in probes:
    try:
        lib.get_by_name(ProbeSeq, p)
    except Exception as e:
        print(f"Failed to find {p} in LIMS")
        continue
    
for p in primers:
    try:
        lib.get_by_name(Primer, p)
    except Exception as e:
        print(f"Failed to find {p} in LIMS")
        continue

In [32]:
emri_comp_name = 'S073.1.5'
ifi44L_comp_name = 'S073.0.2'
ifi44L_wt_name = 'S_IFI44L_WT'
emri_wt_name = 'S_EMR1_ADGRE1_WT'

happy_jaguar_smix = pd.DataFrame(
    [
        ['S073.1.5', 10**3.6, 'copies'],  # EMRI competitor
        ['FP014', 60, 'nM'],  # EMRI primer
        ['RP014', 60, 'nM'],  # EMRI primer
        ['Pr001', 200, 'nM'],  # EMRI competitive probe
        ['Pr007', 200, 'nM'],
        ['S073.0.2', 10**4.5, 'copies'],  # IFI44L competitor
        ['FP017', 70, 'nM'],  # IFI44L primer
        ['RP015', 70, 'nM'],  # IFI44L primer
        ['Pr004', 200, 'nM'],  # IFI44L competitive probe
        ['Pr008', 200, 'nM'],
        ['Taqman Fast Virus Master Mix', 1, 'X']
    ],
    columns=["component", "concentration", "units"],
)
smix = append_smix(happy_jaguar_smix, "happy_jaguar")

The shape of `smix` is now 107 x 4


In [34]:
JG073N_data = process_experiment(
    "JG073N", "JG073N Febrile Signature Gen2.xlsx"
)

JG073_endpoints = pd.read_csv(data_pth / "JG073N Endpoints.csv").rename(
    columns={"SignalDifference": "FAM-HEX"}
)

JG073N_data = JG073N_data.merge(JG073_endpoints[['Experiment', 'Well', "IFI44L Copies", "EMRI Copies", "Material", "ng RNA"]])

JG073N_data = JG073N_data[
    (JG073N_data["IFI44L Copies"] > 0)
    & (JG073N_data["EMRI Copies"] > 0)
    & (JG073N_data["Material"] == "RNA")
    & (JG073N_data["ng RNA"] == 1)
]

JG073N_meta = JG073N_data[["Reaction", "IFI44L Copies", "EMRI Copies"]].melt(
    id_vars=["Reaction"], var_name="component", value_name="concentration"
).assign(
    units = 'copies',
    concentration = lambda df: (10**df.concentration).astype(int),
).replace(
    {
        "component": {
            "IFI44L Copies": ifi44L_wt_name,
            "EMRI Copies": emri_wt_name,
        }
    }
)

JG073N_rxns = JG073N_meta.Reaction.unique()
    
data = append_data(JG073N_data)
meta = append_meta(JG073N_meta, supermix="happy_jaguar")
figs = append_figs(JG073N_rxns, "angry_lemur")
figs = append_figs(JG073N_rxns, "excited_manatee")

The shape of `data` is now 1308 x 9
The shape of `meta` is now 2749 x 4
The shape of `figs` is now 796 x 2
The shape of `figs` is now 796 x 2


# Fig ED1 shy_quokka

In [ ]:
# order_3648138 = {
#     # Order Date 4/21/2022
#     'S075_WT':
#     'S075_SNV':
# }

# order_3650461 = {  
#     # Order Date 4/26/2022 
#     'EGFR L858 Blocker C3':  'GATCACAGATTTTGGGCTGGCCAAACATAA/3SpC3/',
#     'EGFR L858 Blocker DxxDM': ' GATCACAGATTTTGGGCTGGCCAAAC/iSpC3/iSpC3/CA'
# }

order_3399189 = {
    # Order Date 3/2/2021
    'EGFR LNA Blocker C3':  'GATCACAGATTTTGGGC+TGGCCAAACATAA/3SpC3/'
}

order_3186940 = {
    # Order Date 2/5/2020
    'EGFR L858 p0': 'GCAGCATGTCAAGATCACAGATT',
    'EGFR L858 p1': 'ACCTCCTTACTTTGCCTCCTTC',
    'EGFR L858 p2': 'CATTTGCTTCAACAGTGACTACG',
    'EGFR L858 p3': 'GCTATTGCTGGGATTTTGAGG',
    # 'EGFR L858 Blocker C3': 'GATCACAGATTTTGGGCTGGCCAAACATAA/3SpC3/'
}

In [68]:
fragment_sequences['S075_SNV'] = 'GTTGAAGTGGTAGGGATTGCGTTTGTGTGTTCTGCTGTTCATGTGTCGTGCTTCTTTCCTCTAGTTTCCGTCATTGCAAGAACCAAGACCCTCAGGGAGGGAATCACAGTCACTGGGAATCGTCTGGGAACTCTGGCAGTGACTTATGTAGAGAGCAGTTTGGCCCGCCCAAAATCTGTGATCTTGACATGCTGCGATTCGTAACTGGACCGTACTAAAGCAGGATTCAGATTATTGGCCGTTCGTTCTTGTTAGCTCCGTCATTCAAATACCTTACCACATATCCAACCAAAGAGGCAAGTCTCATCAAAGCCCTCCCG'
fragment_sequences['S075_WT'] =  'GTTGAAGTGGTAGGGATTGCGTTTGTGTGTTCTGCTGTTCATGTGTCGTGCTTCTTTCCTCTAGTTTCCGTCATTGCAAGAACCAAGACCCTCAGGGAGGGAATCACAGTCACTGGGAATCGTCTGGGAACTCTGGCAGTGACTTATGTAGAGAGCAGTTTGGCCAGCCCAAAATCTGTGATCTTGACATGCTGCGATTCGTAACTGGACCGTACTAAAGCAGGATTCAGATTATTGGCCGTTCGTTCTTGTTAGCTCCGTCATTCAAATACCTTACCACATATCCAACCAAAGAGGCAAGTCTCATCAAAGCCCTCCCG'
primer_sequences['EGFR L858R Blocker'] = 'GATCACAGATTTTGGGC+TGGCCAAACATAA/3SpC3/'

proud_robin_smix =  pd.DataFrame(
    [
        ['FP0013', 100, 'nM'],
        ['FP001', 100, 'nM'],
        ['EvaGreen', 1, 'X']
    ],
    columns=["component", "concentration", "units"],
)
smix = append_smix(proud_robin_smix, "proud_robin")

humble_scorpion_smix =  pd.DataFrame(
    [
        ['FP0013', 100, 'nM'],
        ['FP001', 100, 'nM'],
        ['FP005', 100, 'nM'],
        ['FP005', 100, 'nM'],
        ['Pr002', 200, 'nM'],
        ['Pr003', 200, 'nM'],
        ['S057.3.2', 100, 'copies'],
        ['S057.4.2', 100, 'copies'],
    ],
    columns=["component", "concentration", "units"],
)
smix = append_smix(humble_scorpion_smix, "humble_scorpion")

The shape of `smix` is now 111 x 4
The shape of `smix` is now 119 x 4


In [71]:
JG075A_data = process_experiment(
    "JG075A", "JG075A EGFR Blocker Stoichiometry.xlsx"
)

# Denote reaction conditions
with np.errstate(divide='ignore'):
    JG075A_data = (
        JG075A_data.merge(
            pd.read_csv(data_pth / "JG075A Plate Map.csv")
        )
        .assign(lg10_Blocker=lambda df: np.log10(df["Blocker μM"]))
        .replace({"lg10_Blocker": {-np.inf: -2}})
        .query("Well != 73")  # Outlier, weird drift in baseline
        .query("Copies >= 10**4")
        .query("Blocker == 'L-MMMMx'")
    )

frags = JG075A_data[['Reaction','Target','Copies']].rename(columns={
    'Target': 'component',
    'Copies': 'concentration',
}).assign(units = 'copies')

blocker = JG075A_data[['Reaction','Blocker μM']].rename(columns={
    'Blocker μM': 'concentration',
}).assign(component = 'EGFR L858R Blocker', units = 'μM')

JG075A_meta = pd.concat([frags, blocker]).sort_values('Reaction')
JG075A_rxns = JG075A_meta.Reaction.unique()

data = append_data(JG075A_data)
meta = append_meta(JG075A_meta, supermix="proud_robin")
figs = append_figs(JG075A_rxns, "proud_robin")
figs = append_figs(JG075A_rxns, "sour_dolphin")

The shape of `data` is now 1378 x 9
The shape of `meta` is now 2959 x 4
The shape of `figs` is now 936 x 2
The shape of `figs` is now 936 x 2


In [82]:
JG075B_data = process_experiment(
    "JG075B", "JG075B L-MMMMx blocker tripartite competition separate.xlsx"
)

with np.errstate(divide='ignore'):
    # Denote reaction conditions
    JG075B_data = (
        JG075B_data.replace(
            {
                "Target": {
                    "S075_WTFAM": "S075_WT",
                    "S075_WTHEX": "S075_WT",
                    "S075_SNVFAM": "S075_SNV",
                    "S075_SNVHEX": "S075_SNV",
                }
            }
        )
        .merge(
            pd.read_csv(data_pth / "JG075B Plate Map.csv")
        )
        .assign(lg10_Blocker=lambda df: np.log10(df["Blocker μM"]))
        .replace({"lg10_Blocker": {-np.inf: -2}})
        .query('Configuration == "2+2"')
        .query('Blocker == "L-MMMMx"')
    )
    
JG075B_data = JG075B_data[(JG075B_data.lg10_Blocker % 0.5).isin([0., np.nan])]

frags = JG075B_data[['Reaction','Target','Copies']].rename(columns={
    'Target': 'component',
    'Copies': 'concentration',
}).assign(units = 'copies')

blocker = JG075B_data[['Reaction','Blocker μM']].rename(columns={
    'Blocker μM': 'concentration',
}).assign(component = 'EGFR L858R Blocker', units = 'μM')

JG075B_meta = pd.concat([frags, blocker]).sort_values('Reaction')
JG075B_rxns = JG075B_meta.Reaction.unique()

data = append_data(JG075B_data)
meta = append_meta(JG075B_meta, supermix="humble_scorpion")
figs = append_figs(JG075B_rxns, "humble_scorpion")
figs = append_figs(JG075B_rxns, "strong_xerus")

The shape of `data` is now 1528 x 9
The shape of `meta` is now 3184 x 4
The shape of `figs` is now 1011 x 2
The shape of `figs` is now 1086 x 2


In [83]:
JG075F_data = process_experiment(
    "JG075F", "JG075F L-MMMMx blocker tripartite competition matrix.xlsx"
)

JG075F_data = (
    JG075F_data.drop(columns=["Sample"]).merge(
        pd.read_csv(data_pth / "JG075F Plate Map.csv"), on="Well"
    )
)

frags = JG075F_data[['Reaction','Target','Copies']].rename(columns={
    'Target': 'component',
    'Copies': 'concentration',
}).assign(units = 'copies')

blocker = JG075F_data[['Reaction','Blocker μM']].rename(columns={
    'Blocker μM': 'concentration',
}).assign(component = 'EGFR L858R Blocker', units = 'μM')

JG075F_meta = pd.concat([frags, blocker]).sort_values('Reaction')
JG075F_rxns = JG075F_meta.Reaction.unique()

data = append_data(JG075B_data)
meta = append_meta(JG075F_meta, supermix="humble_scorpion")
figs = append_figs(JG075F_rxns, "noisy_tapir")

The shape of `data` is now 1528 x 9
The shape of `meta` is now 4676 x 4
The shape of `figs` is now 1459 x 2


# S6 liquid_yabby

Order number 3650428 on 4/26/2022